How many different ways do we find to note the ISO standard and version?

So for any XML reseponse, check for the version.

In [14]:
%reload_ext autoreload
%autoreload 2

import os
import glob
import json
from lxml import etree
from semproc.rawresponse import RawResponse
from semproc.xml_utils import extract_item

def convert_header_list(headers):
    return dict(
        (k.strip().lower(), v.strip()) for k, v in (
            h.split(':', 1) for h in headers)
    )

In [5]:
files = glob.glob('/Users/sparky/Documents/solr_responses/solr_20150922_docs/*.json')

In [6]:
xpaths = [
    (['//*', 'metadataStandardName', 'CharacterString'], ['*', 'metadataStandardVersion', 'CharacterString']),
    (['//*', 'seriesMetadata', 'MD_Metadata', 'metadataStandardName', 'CharacterString'], ['*', 'seriesMetadata', 'MD_Metadata', 'metadataStandardVersion', 'CharacterString']),
    (['//*', 'seriesMetadata', 'MI_Metadata', 'metadataStandardName', 'CharacterString'], ['*', 'seriesMetadata', 'MI_Metadata', 'metadataStandardVersion', 'CharacterString'])
]

In [17]:
versions = set()
for f in files[100000:150000]:
    with open(f, 'r') as g:
        data = json.loads(g.read())
    
    response = data.get('raw_content')
    headers = convert_header_list(data.get('response_headers', []))
    content_type = headers.get('content-type', '')
    
    rr = RawResponse(response, content_type)
    try:
        content = rr.clean_raw_content()
    except:
        print 'FAILED ', f
        continue

    if rr.datatype != 'xml':
        continue
    
    try:
        xml = etree.fromstring(content)
    except:
        continue
        
    for std_name, std_vsn in xpaths:
        sn = extract_item(xml, std_name)
        sv = extract_item(xml, std_vsn)
        
        if sn or sv:
            versions.add('Standard: {0}, Version: {1}'.format(sn, sv))
    

FAILED  /Users/sparky/Documents/solr_responses/solr_20150922_docs/2b727b1afb9381062a3f5dd05802e56ce21eeee8.json
FAILED  /Users/sparky/Documents/solr_responses/solr_20150922_docs/3354a2bd8c334dddcb82c4cf50bc18d8619db25e.json
FAILED  /Users/sparky/Documents/solr_responses/solr_20150922_docs/3459af68928a619ac5cc332e251ff72a62bd10d9.json


In [18]:

versions


{'Standard: ANZLIC Metadata Profile: An Australian/New Zealand Profile of AS/NZS ISO 19115:2005, Geographic information - Metadata, Version: 1.1',
 'Standard: ISO 19115:2003 - Geographic information - Metadata, Version: ISO 19115:2003',
 'Standard: ISO19115, Version: 2003/Cor.1:2006'}